In [4]:
import cv2
import pandas as pd
import numpy as np

FACE RECOGNITION 

In [5]:
#violajones face detection using opencv
def detect_faces(image_path, scaleFactor, minNeighbors, minSize):
    #read image
    image = cv2.imread(image_path)
    #load the pre-trained model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    #convert the image to gray scale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #detect faces
    faces = face_cascade.detectMultiScale(
        gray_image,
        scaleFactor=scaleFactor,
        minNeighbors=minNeighbors,
        minSize=minSize,
    )
    #if no bounding boxes are found, return 0,0,0,0
    if len(faces) == 0:

        return np.array([0,0,0,0])
    #return the bounding boxes
    return faces.flatten()


In [6]:
def compute_iou(box1, box2):
    # determine the (x, y)-coordinates of the intersection rectangle
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[0] + box1[2], box2[0] + box2[2])
    y2 = min(box1[1] + box1[3], box2[1] + box2[3])

    # compute the area of intersection rectangle
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    #compute the area of both the prediction and ground-truth boxes
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]

    # compute the union area
    union_area = box1_area + box2_area - inter_area
    #return IoU
    return inter_area / union_area if union_area > 0 else 0


In [7]:
def evaluate_model(dataset, scaleFactor, minNeighbors, minSize):
    #iou scores for each sample
    iou_scores = []
    for item in dataset["identity"].unique():
        #get all the samples for a particular identity
        df = dataset[dataset["identity"] == item]
        
        #for each sample of the same identity
        for i in range(len(df)):            
            path = "data/" + str(df.iloc[i]["idx"]) + ".jpg"
             # Ground truth bounding boxes
            gt_box = [df.iloc[i]["x_1"], df.iloc[i]["y_1"], df.iloc[i]["width"], df.iloc[i]["height"]] 
            
            # Detected bounding boxes with viola-jones
            detected_face = detect_faces(path, scaleFactor, minNeighbors, minSize)
            
            #compute iou
            iou = compute_iou(gt_box, detected_face)
            
                
        iou_scores.append(iou)

    avg_iou = np.mean(iou_scores) if iou_scores else 0
    return avg_iou

In [8]:
def optimize_parameters(dataset):
    best_params = {}
    best_iou = 0
    #grid search for best parameters
    for scaleFactor in [1.1,1.2,1.3 ,1.4]:
        for minNeighbors in range(3, 6):
            for minSize in [(20, 20), (30, 30)]:
                avg_iou = evaluate_model(dataset, scaleFactor, minNeighbors, minSize)
                if avg_iou > best_iou:
                    best_iou = avg_iou
                    #dict of best parameters
                    best_params = {
                        "scaleFactor": scaleFactor,
                        "minNeighbors": minNeighbors,
                        "minSize": minSize,
                    }
    return best_params, best_iou

In [7]:
def test_model(dataset, best_params):
    #test the model with the optimized parameters, calc avg iou
    avg_iou = evaluate_model(
        dataset,
        best_params["scaleFactor"],
        best_params["minNeighbors"],
        best_params["minSize"],
    )
    print(f"Average IoU on test set: {avg_iou}")

In [8]:
# Load the dataset
df = pd.read_csv('CelebA-HQ-small.csv')
# Get the train split
df_test = df.loc[df["split"] == "train"]
# train the model
best_params, best_iou =  optimize_parameters(df_test)


In [9]:

print(f"Best parameters: {best_params}, Best IoU: {best_iou}")
# Get the test split
df_test = df.loc[df["split"] == "test"]
# test the model
test_model(best_params=best_params, dataset=df_test)

Best parameters: {'scaleFactor': 1.2, 'minNeighbors': 3, 'minSize': (20, 20)}, Best IoU: 0.6754693822947886
Average IoU on test set: 0.5900289445468367


FEATURE EXTRACTION

In [9]:
def extract_all_feature_vectors(dataset,function, n_points=8, radius=1, grid_size=(4,4), method="uniform", scaleFactor = 1.3, minNeighbors=3, minSize=(20,20), useBoundingBox=True):
    feature_vectors = []
    image_ids = []
    for i in range(len(dataset)):   
        path = "data/" + str(df.iloc[i]["idx"]) + ".jpg"
        # Detected bounding boxes with viola-jones
        detected_face = detect_faces(path, scaleFactor, minNeighbors, minSize)
        image_ids.append(df.iloc[i]["idx"])
        if function == "lbp":
            feature_vectors.append(extract_features_lbp(path, detected_face, n_points, radius, grid_size, method, useBoundingBox=useBoundingBox))
        elif function == "hog":
            feature_vectors.append(extract_features_hog(path, detected_face, useBoundingBox=useBoundingBox))
    return feature_vectors, image_ids



In [10]:
def chi2_distance(A, B):
 
    chi = 0.5 * np.sum([((a - b) ** 2) / (a + b + 1e-8) 
                      for (a, b) in zip(A, B)])
    return chi

In [11]:
def get_genuines_impostors_scores(dataset,function,n_points=8, radius=1, grid_size=(4,4), method="uniform", scaleFactor = 1.3, minNeighbors=3, minSize=(20,20),useBoundingBox=True):
    #extract feature vectors for all the samples
    genuines = []
    impostors = []
    feature_vectors,image_ids = extract_all_feature_vectors(dataset,function,n_points=n_points, radius=radius, grid_size=grid_size, method=method, scaleFactor = scaleFactor, minNeighbors=minNeighbors, minSize=minSize,useBoundingBox=useBoundingBox)
    for feature_vector, image_id in zip(feature_vectors, image_ids):
        #get the identity of the sample
        identity = dataset[dataset["idx"] == image_id]["identity"].values[0]
        #get all the samples of the same identity
        df_gen = dataset[dataset["identity"] == identity]
        #get all the samples of different identity
        df_imp = dataset[dataset["identity"] != identity]
        for i in range(len(df_gen)):
            #compute the chi2 distance between the feature vectors
            genuines.append(chi2_distance(feature_vector, feature_vectors[i]))
            
        for i in range(len(df_imp)):
            #compute the chi2 distance between the feature vectors
            impostors.append(chi2_distance(feature_vector, feature_vectors[i]))
    return genuines, impostors
        

    

In [12]:
def classification_accuracy(genuines, impostors, start_treshold, end_treshold, step):
    classification_acc = []
    prec = []
    recall = []
    f1 = []

    for treshold in range(start_treshold, end_treshold, step):
        correctly_classified_gen = 0
        correctly_classified_imp = 0
        #count number of correctly classified genuines
        for el in genuines:
            if el <= treshold:
                correctly_classified_gen+=1
        #count number of correctly classified impostors
        for el in impostors:
            if el > treshold:
                correctly_classified_imp+=1
        
        classification_acc.append((correctly_classified_gen + correctly_classified_imp)/(len(genuines) + len(impostors)))
        prec.append((correctly_classified_gen)/(correctly_classified_gen + len(impostors) - correctly_classified_imp))
        recall.append(correctly_classified_gen/len(genuines))
        f1.append((2*prec[-1]*recall[-1])/(prec[-1] + recall[-1] + 0.00000001))
    
    
    indeks = f1.index(max(f1))
    best_treshold = start_treshold + indeks*step

    return best_treshold, classification_acc[indeks], prec[indeks], recall[indeks], f1[indeks]


In [13]:
def plot_curve(genuine, impostors):
    genuine_frequency= []
    genuine_count = []


    impostors_frequency= []
    impostors_count = []

    #create frequency and count
    for i in range(0, int(max(genuine))+1):
        genuine_frequency.append(genuine.count(i))
        genuine_count.append(i)

        #same for impostors
    for i in range(0, int(max(impostors))+1):
        impostors_frequency.append(impostors.count(i))
        impostors_count.append(i)

    plt.plot(genuine_count, genuine_frequency,label="genuines")
    plt.plot(impostors_count, impostors_frequency, label="impostors")

    plt.legend()
    plt.show()


LBP

In [14]:
from skimage.feature import local_binary_pattern


In [106]:
def extract_features_lbp(image_path, bounding_box, n_points=8, radius=1 ,grid_size=(8,8), useBoundingBox=True):
    #read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    face_region = image[bounding_box[1] : bounding_box[1] + bounding_box[3],
        bounding_box[0] : bounding_box[0] + bounding_box[2]]
    
    if bounding_box.all() and useBoundingBox:
        lbp = local_binary_pattern(face_region, n_points, radius, method="uniform")
    else:
        lbp = local_binary_pattern(image, n_points, radius, method="uniform")   

    lbp = cv2.resize(lbp, (256, 256))   

    features = []
    bins = n_points + 2

    height, width = lbp.shape

    for wx in range(0,width,grid_size[0]):
        for wy in range(0,height,grid_size[1]):
            hist = np.zeros(bins)
            for x in range(wx, min(wx + grid_size[0], width)):
                for y in range(wy, min(wy + grid_size[1], height)):
                    hist[int(lbp[y, x])] += 1
            features.extend(hist)

    return np.array(features), lbp
    

In [104]:
image_path = "data/123.jpg"
face_box  = detect_faces(image_path, 1.3, 3, (20,20))

feature1,lbp = extract_features_lbp(image_path, face_box, n_points=24, radius=3, grid_size=(4,4))
#cv2.imshow("lbp",lbp)
#cv2.waitKey(0)

In [87]:
image_path = "data/285.jpg"
face_box  = detect_faces(image_path, 1.3, 3, (20,20))

feature2,lbp = extract_features_lbp(image_path, face_box, n_points=24, radius=3, grid_size=(4,4), useBoundingBox=True)
#cv2.imshow("lbp",lbp)
#cv2.waitKey(0)

In [90]:
print(len(feature1), len(feature2))
chi2_distance(feature1, feature2)

106496 106496


np.float64(32166.05206428853)

In [107]:
df = pd.read_csv('CelebA-HQ-small.csv')
df = df.iloc[::10]
lbp_genuines,lbp_impostors = get_genuines_impostors_scores(df, "lbp")

TypeError: extract_features_lbp() got multiple values for argument 'useBoundingBox'

In [15]:
# Load the dataset 
dataset = pd.read_csv('CelebA-HQ-small.csv')
lbp_genuines,lbp_impostors = get_genuines_impostors_scores(dataset, "lbp")

Histogram of oriented gradients - HOG

In [115]:
from skimage.feature import hog
from skimage import io
import matplotlib.pyplot as plt

In [142]:
def extract_features_hog(image_path, bounding_box,orientatins=9, piexels_per_cell = (8,8), cells_per_block = (2,2), image_size=(256,256), useBoundingBox=True):
    image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)

    
    face_region = image[bounding_box[1] : bounding_box[1] + bounding_box[3],
        bounding_box[0] : bounding_box[0] + bounding_box[2]]
    
    
    if bounding_box.all() and useBoundingBox:
        #face_region = cv2.resize(face_region, image_size)
        features,im = hog(face_region, orientations=orientatins, pixels_per_cell=piexels_per_cell, cells_per_block=cells_per_block, visualize=True)
    else:
        #image = cv2.resize(image, image_size)
        features,im = hog(image, orientations=orientatins, pixels_per_cell=piexels_per_cell, cells_per_block=cells_per_block, visualize=True)


    return features,im


In [148]:
image_path = "data/123.jpg"
face_box  = detect_faces(image_path, 1.3, 3, (20,20))

feature1,im = extract_features_hog(image_path, face_box)


In [144]:
image_path = "data/285.jpg"
face_box  = detect_faces(image_path, 1.3, 3, (20,20))

feature2,im = extract_features_hog(image_path, face_box)

In [149]:
cv2.imshow("hog",im)
cv2.waitKey(0)

-1

In [146]:
chi2_distance(feature1, feature2) 

np.float64(4129.662701870718)

In [18]:
hog_genuines, hog_impostors = get_genuines_impostors_scores(dataset, "hog")

In [29]:
print(np.mean(hog_genuines), np.mean(hog_impostors))
np.mean(lbp_genuines), np.mean(lbp_impostors)

226.10225245618656 242.87136455831742


(np.float64(1.5045868453550197), np.float64(1.1848743694056472))

DENSE SIFT

In [ ]:
image_path = "data/112.jpg"
# Load and preprocess the image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

boxes = detect_faces(image_path, 1.3, 3, (20, 20))

image = image[boxes[1] : boxes[1] + boxes[3], boxes[0] : boxes[0] + boxes[2]]
# Parameters for Dense SIFT
grid_spacing = 8  # Distance between grid points
patch_size = 16   # Size of the patch for each descriptor

# Initialize SIFT
sift = cv2.SIFT_create()

# Generate grid points
keypoints = []
h, w = image.shape
for y in range(0, h, grid_spacing):
    for x in range(0, w, grid_spacing):
        keypoints.append(cv2.KeyPoint(x, y, patch_size))

# Compute Dense SIFT descriptors
keypoints, descriptors = sift.compute(image, keypoints)

# Visualize grid points
image_with_keypoints = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(image_with_keypoints, cmap="gray")
plt.title("Dense SIFT Keypoints")
plt.show()

# Print descriptor shape
print(f"Number of descriptors: {len(descriptors)}")
print(f"Descriptor size: {descriptors.shape}")

In [124]:
image_path = "data/112.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


face_box = detect_faces(image_path, 1.2, 3, (20, 20))
face = image[face_box[1]:face_box[1] + face_box[3], face_box[0]:face_box[0] + face_box[2]]

#fixed_face = cv2.resize(face, (64, 64))
# Parameters for HOG
pixels_per_cell = (8, 8)
cells_per_block = (2, 2) 
orientations = 9 #bin size for the histogram

# Compute HOG features and HOG image
hog_features = hog(
    face,
    orientations=orientations,
    pixels_per_cell=pixels_per_cell,
    cells_per_block=cells_per_block,

)

# Display the HOG image
#plt.figure(figsize=(8, 8))
#plt.axis("off")
#plt.title("HOG Visualization")
#plt.imshow(hog_image, cmap="gray")
#plt.show()

# HOG features vector
print(f"HOG feature vector size: {hog_features.shape}")


HOG feature vector size: (298116,)


In [ ]:
#read image
image_path = "data/112.jpg"
image = cv2.imread(image_path)
#convert the image to gray scale
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

bounding_box = detect_faces(image_path, 1.2, 3, (20, 20))

grid_size = (8, 8)
n_points = 8
radius = 1
method = "uniform"

face_region = image[bounding_box[1] : bounding_box[1] + bounding_box[3],
    bounding_box[0] : bounding_box[0] + bounding_box[2]]
if bounding_box.all():
    lbp = local_binary_pattern(face_region, n_points, radius, method=method)
else:
    lbp = 255*np.ones((face_region.shape[0], face_region.shape[1]))

features = []

# Divide the image into grids
h, w = lbp.shape
grid_h, grid_w = h // grid_size[0], w // grid_size[1]
print(grid_h, grid_w)

for i in range(grid_size[0]):  # Rows
    for j in range(grid_size[1]):  # Columns
        # Extract grid region
        grid = lbp[i * grid_h:(i + 1) * grid_h, j * grid_w:(j + 1) * grid_w]
        
        # Compute histogram
        hist, _ = np.histogram(grid.ravel(), bins=n_points+3, range=(0, n_points + 2))
        

        # Normalize histogram
        hist = hist / (np.sum(hist) + 1e-8)
        
        
        # Append to feature vector
        features.extend(hist)

print(np.array(features).shape)
print(features)
print(lbp.shape)


92 92
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
(704,)
[np.float64(0.055056710774982205), np.float64(0.07951323251408374), np.float64(0.06758034026457044), np.float64(0.12051039697528294), np.float64(0.21620982986741938), np.float64(0.11862003780704321), np.float64(0.06982514177685512), np.float64(0.08187618147438341), np.float64(0.07455103969745444), np.float64(0.11625708884674354), np.float64(0.0), np.float64(0.03414461247633017), np.float64(0.06769848771258542), np.float64(0.0621455576558812), np.float64(0.12653591682404708), np.float64(0.2655954631376824), np.float64(0.14756616257071412), np.float64(0.08034026465018862), np.float64(0.06876181474472026), np.float64(0.0600189035916115), np.float64(0.08719281663505765), np.float64(0.0), np.float64(0.03438090737236013), np.fl

In [78]:
np.arange(0, n_points + 3)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [45]:



# Load the image 
image_path = "data/112.jpg"
image = cv2.imread(image_path)


HEIGHT = image.shape[0]
WIDTH = image.shape[1]

faces = detect_faces(image_path, 1.2, 6, (40, 40))
print(faces.shape)

box_annot = df.loc[df['idx'] == int(image_path.split('/')[-1].split('.')[0]), ['x_1', 'y_1', 'width', 'height']].values
box_annot = box_annot.flatten()


print(compute_iou(faces, box_annot))

cv2.rectangle(image, (faces[0], faces[1]), (faces[0]+faces[2], faces[1]+faces[3]), (255, 0, 0), 2)
cv2.rectangle(image, (box_annot[0], box_annot[1]), (box_annot[0]+box_annot[2], box_annot[1]+box_annot[3]), (0, 255, 255), 2)

# Display the image
cv2.namedWindow("Detected Faces", cv2.WINDOW_NORMAL) 
cv2.resizeWindow("Detected Faces", int(WIDTH*0.8), int(0.8*HEIGHT))
cv2.imshow("Detected Faces", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


(4,)
0.7108736390202173
